<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Other-information" data-toc-modified-id="Other-information-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Other information</a></span></li><li><span><a href="#Model-environment-set-up" data-toc-modified-id="Model-environment-set-up-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Model environment set up</a></span><ul class="toc-item"><li><span><a href="#abiotic-factors:-solar,-temperature-cycles" data-toc-modified-id="abiotic-factors:-solar,-temperature-cycles-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>abiotic factors: solar, temperature cycles</a></span></li><li><span><a href="#ODE-model:-Generalised-Logistic" data-toc-modified-id="ODE-model:-Generalised-Logistic-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>ODE model: Generalised Logistic</a></span></li><li><span><a href="#default-parameters" data-toc-modified-id="default-parameters-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>default parameters</a></span></li><li><span><a href="#test-parameter-ranges" data-toc-modified-id="test-parameter-ranges-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>test parameter ranges</a></span></li></ul></li></ul></div>

# Other information
[main notebook](https://nbviewer.jupyter.org/github/ph-u/Project/blob/master/meeting/MRes_02.ipynb)
[single Factor trial](https://nbviewer.jupyter.org/github/ph-u/Project/blob/master/meeting/MRes_02_sFac.ipynb)
# Model environment set up

In [1]:
using RCall, PyCall, Plots, ProgressMeter, DataFrames
mt = pyimport("math")
sc = pyimport("scipy")
itg = pyimport("scipy.integrate")
R"cbp<-c('#000000', '#E69F00', '#56B4E9', '#009E73', '#0072B2', '#D55E00', '#CC79A7', '#e79f00', '#9ad0f3', '#F0E442', '#999999', '#cccccc', '#6633ff', '#00FFCC', '#0066cc')"

RObject{StrSxp}
 [1] "#000000" "#E69F00" "#56B4E9" "#009E73" "#0072B2" "#D55E00" "#CC79A7"
 [8] "#e79f00" "#9ad0f3" "#F0E442" "#999999" "#cccccc" "#6633ff" "#00FFCC"
[15] "#0066cc"


## abiotic factors: solar, temperature cycles

In [2]:
function solar_dy(t, flux)
    S_dy = flux * mt.sin(2 * mt.pi / (24) * t)
    # add daily fluctuation (unit = hr)
    return (S_dy)
end
function solar_yr(t, flux)
    S_yr = flux * mt.sin(2 * mt.pi / (365.25 * 24) * t)
    # add seasonal fluctuation (unit = hr)
    return (S_yr)
end
function solar_sc(t, flux)
    S_sc = flux * mt.sin(2 * mt.pi / (11.8 * 365.25 * 24) * t)
    # add solar cycle fluctuation (unit = hr)
    return (S_sc)
end
function solar_al(t, base, f_sc, f_yr, f_dy)
    S_al = solar_sc(t, f_sc) + solar_yr(t, f_yr) + solar_dy(t, f_dy) + base
    ## three cycles additive effect
    return (S_al)
end

solar_al (generic function with 1 method)

## ODE model: Generalised Logistic
modified a bit in order also to plot solar cycle

In [3]:
function ebc_GLc(pops, t, Tk, TbS, TbT, CharP, CharQ)
    
    ## characteristics segregation
    p = pops[:1] # population of p
    q = pops[:2] # population of q
    rp=CharP[:1]+1; kp=CharP[:2]; Jfp=CharP[:3]; Tkp=CharP[:4]
    rq=CharQ[:1]+1; kq=CharQ[:2]; Jfq=CharQ[:3]; Tkq=CharQ[:4]
    
    ## abiotic factors
    SS = solar_al(t, TbS[:1], TbS[:2], TbS[:3], TbS[:4]) ## solar budget
    ST = solar_al(t, TbT[:1], TbT[:2], TbT[:3], TbT[:4]) ## temperature threat level
    
    ## energy budget
    Jp = SS*Jfp
    Jq = SS*Jfq
    # assume solar spectrum unchange throughout the day
    
    ## p & q filling up the space, so hpq + hqp = 1
    hpq = Jp*p/(Jp*p +Jq*q) # hindrance from q on p
    hqp = 1-hpq # hindrance from p on q
    
    ## thermal kill ratio
    if ST < Tk # assume if temperature lower than threshold, fixed ratio of population die
        TRp = Tkp
        TRq = TKq
    else
        TRp = 0
        TRq = 0
    end
    
    ## ODEs
    dpdt = p*(rp*kp -rp*p -rp*hpq*q -TRp) # +growth*carrying -intra-competition -hinder by q -thermal stress (when applicable)
    dqdt = q*(rq*kq -rq*q -rq*hqp*p -TRq) # +growth*carrying -intra-competition -hinder by p -thermal stress (when applicable)
    
    return (dpdt, dqdt, SS-TbS[:1]+.5)
end

ebc_GLc (generic function with 1 method)

## default parameters

In [4]:
## unknown reason, must reload for every simulation
pops = sc.array([.5, .3, 0]) # initial population p, q

t = sc.linspace(0, 10, 10000) # time series (hours)

Tk = 5 # thermal kill activation threshold

TbS = sc.array([1500, 1, 100, 30]) # solar: mean per unit time energy, solar cycle amplitude, annual amplitude, daily amplitude

TbT = sc.array([15, 0, 10, 3]) # temperature: mean per unit time, solar cycle amplitude, annual amplitude, daily amplitude

CharP = sc.array([.04, .7, .3, .1]) # p characteristics: net growth rate, carrying cap ratio, energy extraction fraction, thermal sensitive fraction

CharQ = sc.array([.04, .7, .3, .1]) # q characteristics: net growth rate, carrying cap ratio, energy extraction fraction, thermal sensitive fraction

println()

## test parameter ranges

In [ ]:
## parameter test trial dump
ppdef = [.9, .8, .7, .6, .5, .4, .3, .2, .1]
rpdef = [.07, .06, .05, .04, .03, .02, .01]
kpdef = [1., .9, .8, .7, .6, .5, .4]
jpdef = [.7, .6, .4, .3, .2, .1, .05]
tpdef = [.6, .5, .3, .1, .05, .02, .01]

println()